In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
!cp -r '/content/drive/MyDrive/Touche clef/trainingset-ideology-power.zip' '/content/'

In [ ]:
import pandas as pd
import zipfile
import io
import os

# Path to the zip file
zip_file_path = '/content/trainingset-ideology-power.zip'

# Function to read TSV files from a specific folder within a zip file and return a DataFrame
def read_tsv_from_zip_folder(zip_file_path, folder_name):
    dataframes = []
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for file_name in zip_ref.namelist():
            if file_name.startswith(folder_name) and file_name.endswith('.tsv'):
                with zip_ref.open(file_name) as file:
                    # Read TSV file into DataFrame
                    df = pd.read_csv(io.TextIOWrapper(file, 'utf-8'), sep='\t')
                    # Append DataFrame to the list
                    dataframes.append(df)
    # Concatenate all DataFrames into one
    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df

# Specify the folder name you want to extract files from
specific_folder_name = 'power'

# Call the function to read TSV files from the specified folder within the zip file
df = read_tsv_from_zip_folder(zip_file_path, specific_folder_name)

# Now, df contains the combined DataFrame from all TSV files in the specified folder within the zip file


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Load data  # Replace 'your_data.csv' with your data file path
df=df.dropna()
# Prepare labels
train_labels = df['label']

# Split data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(
    df['text_en'], train_labels, test_size=0.20, random_state=42
)
train_data  = df['text_en']
train_labels = df['label']
# Replace NaN values in 'text_en' column with empty string in training and test data
train_data = train_data.fillna('')
val_data = val_data.fillna('')

# Define TfidfVectorizer for text data
vectorizer = TfidfVectorizer(
    max_features=10000, ngram_range=(1, 2), min_df=2, max_df=0.9, stop_words='english'
)

# Define class weights based on training labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_dict = dict(enumerate(class_weights))

# Create a pipeline with TfidfVectorizer and LogisticRegression model
pipeline = make_pipeline(vectorizer, LogisticRegression(class_weight=class_weights_dict, random_state=42))

# Fit the model on training data
pipeline.fit(train_data, train_labels)

# Predict on the validation set
val_predictions = pipeline.predict(val_data)

# Evaluate the model using F1 score
val_f1 = f1_score(val_labels, val_predictions)
print(f"Validation F1 Score: {val_f1}")

# Print classification report
print("\nClassification Report:")
print(classification_report(val_labels, val_predictions))

# Predict on the test data
test_data = df['text_en']
test_data = test_data.fillna('')

test_predictions = pipeline.predict(test_data)

# Create a DataFrame with the predictions
output_df = pd.DataFrame({'id': df['id'], 'label': test_predictions})

# Save the DataFrame to a CSV file
output_df.to_csv('predictions.csv', index=False)
print("Predictions saved to predictions.csv")


In [ ]:
val_f2 = f1_score(df['label'], test_predictions)
print(val_f2)

In [ ]:
import joblib
joblib.dump(pipeline, 'power logistic f1-77.joblib')
print("Model saved to best_model.joblib")

In [ ]:
import zipfile

# Specify the path to the zip file
zip_file_path = '/content/ideology-power-st-testset.zip'

# Specify the directory where you want to extract the files
extract_dir = 'content/'

# Create a ZipFile object
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents of the zip file
    zip_ref.extractall(extract_dir)

print("Files extracted successfully to", extract_dir)

In [ ]:
import pandas as pd

# Path to the TSV file
file_path = '/content/content/power/power-es-test.tsv'
fpath='pixel-power-es-run1.tsv'

# Read the TSV file into a DataFrame
df = pd.read_csv(file_path, sep='\t')

# Display the DataFrame
print(df)


In [ ]:
import pandas as pd
import joblib  # Import joblib for loading the model
all="ua"
fpath="pixel-power-"+all+"-run2.tsv"
file_path = "/content/content/power/power-"+all+"-test.tsv"

# Read the TSV file into a DataFrame
df = pd.read_csv(file_path, sep='\t')

# Load the saved model using joblib # Replace 'best_model.joblib' with the path to your saved model file
# Load test data
test_data = df  # Replace 'your_test_data.csv' with your test data file path

# Replace NaN values in 'text_en' column with empty string in test data
test_data['text_en'] = test_data['text_en'].fillna('')

# Predict on the test data using the loaded model
test_predictions = model.predict(test_data['text_en'])

# Create a DataFrame with the predictions
output_df = pd.DataFrame({'id': test_data['id'], 'label': test_predictions})

# Save the DataFrame to a CSV file
output_df.to_csv(fpath, sep='\t', columns=['id', 'label'], header=False, index=False)
print("Predictions saved to ",fpath)